# Recurrent Neural Networks for sequential recommendation

The purpose of this notebook is to create an RNN architecture capable of addressing our problem in a more precise and computationally efficient manner.

In [ ]:
from keras.layers import Dense, Activation, Input, LSTM, Embedding, TimeDistributed
from keras.models import load_model, Model
from keras.utils import Sequence

import matplotlib.pyplot as plt
import csv
import math
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Designing the model

In [ ]:
# Nombre de valeurs dans le dictionnaire + la valeur vide
d = 14370 + 1 # 1 catégories supplémentaires : une <EOS>
# Taille de l'input
Tx = 64
# Taille de l'output
Ty = 16
# Batch Size
m = 1500
# Dimension de l'état caché du LSTM de l'encodeur
n_e = 256
# Dimension de l'état caché du LSTM du décodeur
n_d = 256

In [ ]:
def RNN_model_with_embeddings():
    emb_model = load_model("/content/drive/MyDrive/PSC Recommandation séquentielle/Modèles/RNN/Embbeding_64 save")
    embedding_layer = Embedding(d , 64, weights = [emb_model.get_weights()[0]], trainable = True)

    encoder_inputs = Input(shape=(Tx), dtype='int32',)
    encoder_LSTM = LSTM(n_e, return_state=True, name="encoder_LSTM")
    embedded_encoder_inputs = embedding_layer(encoder_inputs)
    encoder_outputs, state_h, state_c = encoder_LSTM(embedded_encoder_inputs)

    decoder_inputs = Input(shape=(Ty), dtype='int32',)
    decoder_LSTM = LSTM(n_d, return_state=True, return_sequences=True, name="decoder_LSTM")
    embedded_decoder_inputs = embedding_layer(decoder_inputs)
    decoder_outputs, _, _ = decoder_LSTM(embedded_decoder_inputs, initial_state=[state_h, state_c])

    outputs = TimeDistributed(Dense(d, activation='softmax'))(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], outputs)

    return model

In [ ]:
#model = RNN_model_with_embeddings()
#model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics= "accuracy")
#model.summary()

In [ ]:
#model.save("/content/drive/MyDrive/PSC Recommandation séquentielle/Modèles/RNN/RNN_3_save")

## Model training

In [ ]:
folder = "/content/drive/MyDrive/PSC Recommandation séquentielle/Données/DataTables/"

class DataGenerator(Sequence):
  def __init__(self , nb_lines, X_path, Y_path):
    self.X_path = X_path
    self.X_reader = csv.reader(open(folder + X_path , "r"))
    self.Y_path = Y_path
    self.Y_reader = csv.reader(open(folder + Y_path , "r"))
    self.nb_lines = nb_lines

  def __len__(self):
    return math.ceil(self.nb_lines/m)

  def __getitem__(self, idx):
    X1 = []
    X2 = []
    Y = []
    for i in range(m):
      x,y = self.getNextSample()
      x = [int(i) for i in x]
      y = [int(i) for i in y]
      X1.append(x)
      X2.append([0] + y[0:Ty-1])
      Y.append(y)
    X1 = np.array(X1)
    X2 = np.array(X2)
    return [np.array(X1), np.array(X2)] , np.array(Y)

  def getNextSample(self):
    x = next(self.X_reader , None)
    y = next(self.Y_reader , None)
    if x is None:
      self.X_reader = csv.reader(open(folder + self.X_path , "r"))
      self.Y_reader = csv.reader(open(folder + self.Y_path , "r"))
      x = next(self.X_reader , None)
      y = next(self.Y_reader , None)
    return x , y

In [ ]:
model = load_model("/content/drive/MyDrive/PSC Recommandation séquentielle/Modèles/RNN/RNN_3_save")
model.compile(optimizer="Adamax", loss="sparse_categorical_crossentropy", metrics= "accuracy")
model.summary()

In [ ]:
#checkpoint = ModelCheckpoint("/content/drive/MyDrive/PSC Recommandation séquentielle/Modèles/RNN/RNN model 3_64 save/saved_model.pb",
#                             monitor='loss', verbose=1, save_weights_only = True,
#                             save_best_only=False, mode='auto', save_freq=1000)

train_gen = DataGenerator(3705954
                          , "reversed_X_train.csv"
                          ,"Y_train.csv")
model.fit(train_gen , epochs=4, verbose = 1, #callbacks=[checkpoint]
          )

model.save("/content/drive/MyDrive/PSC Recommandation séquentielle/Modèles/RNN/RNN_3_save")


### Hyperparameter tuning

## Results

### Beam search inference

In [ ]:
def beam_search(input_sequence , k):
  # On utilise une entrée bidon pour nourir le modèle et obtenir le premier item
  X2 = [0]*Ty
  input_sequence = np.array([input_sequence])
  # On cherche à savoir les différentes probabilité que le premier item soit acheté
  Y = model.predict([input_sequence , np.array([X2])] , batch_size=1)
  Y = [(Y[0][0][i] , i) for i in range(2,d)]
  Y.sort(reverse = True)

  K_best_candidates = []
  for i in range(k):
    score , item = Y[i]
    K_best_candidates.append((score , [item]))

  for i in range(1, Ty):
    next_best_candidates = []
    for candidat in K_best_candidates :
      score , X2 = candidat
      X = X2.copy() + [0]*(Ty - len(X2))
      Y = model.predict( [input_sequence , np.array([X])] , batch_size=1)
      Y = [(Y[0][i][j] , j) for j in range(1, d)]
      Y.sort(reverse = True)
      for j in range(k):
        proba , item = Y[j]
        next_best_candidates.append((score*proba , X2.copy() + [item]))

    next_best_candidates.sort(reverse= True)
    K_best_candidates = next_best_candidates[0:k]

  _ , infered_sequence = K_best_candidates[0]

  return [item for item in infered_sequence]



with open(folder + "Y_dev.csv" , "r") as csvfile:
  r = csv.reader(csvfile)
  for i in range(0) : next(r)
  T = [int(i) for i in next(r)]
  print("Séquence de référence: ")
  print(T)

with open(folder + "reversed_X_dev.csv" , "r") as csvfile:
  r = csv.reader(csvfile)
  for i in range(0) : next(r)
  T = beam_search([int(i) for i in next(r)] , 10)
  print("Séquence inférée: ")
  print(T)

### Auto-excitement phenomenon

In [ ]:
  def proba_a_posteriori(input_sequence , k):
    X = np.array([input_sequence])
    Y = beam_search(input_sequence , k)
    X2 = np.array([[0] + Y[0:Ty-1]])
    Y_soft = model.predict([X,X2] , batch_size=1)

    return [Y_soft[0][i][Y[i]] for i in range(Ty)]

In [ ]:
with open(folder + "Y_dev.csv" , "r") as csvfile:
  r = csv.reader(csvfile)
  for i in range(0) : next(r)
  T = [int(i) for i in next(r)]
  print("Séquence de référence: ")
  print(T)

with open(folder + "reversed_X_dev.csv" , "r") as csvfile:
  r = csv.reader(csvfile)
  for i in range(0) : next(r)
  seq = next(r)
  T = beam_search([int(i) for i in seq] , 10)
  print("Séquence inférée: ")
  print(T)
  T_soft = proba_a_posteriori([int(i) for i in seq] , 10)
  print("Proba a posteriori de la séquence :")
  plt.plot(T_soft)

### Removing redundancies

In [ ]:
def beam_search_without_redundance(input_sequence , k):
  # On utilise une entrée bidon pour nourir le modèle et obtenir le premier item
  X2 = [0]*Ty
  input_sequence = np.array([input_sequence])
  # On cherche à savoir les différentes probabilité que le premier item soit acheté
  Y = model.predict([input_sequence , np.array([X2])] , batch_size=1)
  Y = [(Y[0][0][i] , i) for i in range(2,d)]
  Y.sort(reverse = True)

  K_best_candidates = []
  for i in range(k):
    score , item = Y[i]
    K_best_candidates.append((score , [item]))

  for i in range(1, Ty):
    next_best_candidates = []
    for candidat in K_best_candidates :
      score , X2 = candidat
      X = X2.copy() + [0]*(Ty - len(X2))
      Y = model.predict( [input_sequence , np.array([X])] , batch_size=1)
      Y = [(Y[0][i][j] , j) for j in range(1, d)]
      Y.sort(reverse = True)
      j = 0
      a = 0
      while a < k:
        proba , item = Y[j]
        if item not in X:
          next_best_candidates.append((score*proba , X2.copy() + [item]))
          a += 1
        j += 1

    next_best_candidates.sort(reverse= True)
    K_best_candidates = next_best_candidates[0:k]

  _ , infered_sequence = K_best_candidates[0]

  return [item for item in infered_sequence]

In [ ]:
with open(folder + "Y_dev.csv" , "r") as csvfile:
  r = csv.reader(csvfile)
  for i in range(0) : next(r)
  T = [int(i) for i in next(r)]
  print("Séquence de référence: ")
  print(T)

with open(folder + "reversed_X_dev.csv" , "r") as csvfile:
  r = csv.reader(csvfile)
  for i in range(0) : next(r)
  seq = next(r)
  T = beam_search([int(i) for i in seq] , 3)
  print("Séquence inférée: ")
  print(T)
  T_diff = beam_search_without_redundance([int(i) for i in seq] , 3)
  print("Séquence inférée sans redondance :")
  print(T_diff)

  print("Historique :")
  print(seq)